<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/high_kappa_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
high_kappa = pd.read_csv(r'/content/train_data_highkappa.csv')


test = pd.read_csv(r'/content/test_data.csv')

In [ ]:
high_kappa['Target'] = high_kappa['Target'].replace(['Others'],'Invalid')


In [ ]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)
high_kappa_X_tfidf = tfidf_vect.fit_transform(high_kappa['Sentence'])


In [ ]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [ ]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [ ]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [ ]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [ ]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [ ]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(high_kappa_X_tfidf,high_kappa['Target'],test_size=0.26,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6614583333333334
                  precision    recall  f1-score   support

        Analysis       0.51      0.40      0.45       104
      Conclusion       0.58      0.46      0.51        41
           Facts       0.74      0.87      0.80       416
         Invalid       0.49      0.54      0.52       109
           Issue       0.73      0.35      0.47        55
Rule/Law/Holding       0.50      0.21      0.30        43

        accuracy                           0.66       768
       macro avg       0.59      0.47      0.51       768
    weighted avg       0.65      0.66      0.64       768



In [ ]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 3, 2, 3, 2, 2, 3, 0, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2]
[0.9979719319535413, 0.9967494080317273, 0.9896320066984451, 0.9911131515151707, 0.9983674315893399, 0.9941498267380551, 0.9959839513443691, 0.991380395318636, 0.997927658423435, 0.988161665167309, 0.9954043880849256, 0.995458353546969, 0.9973038330488185, 0.9966649749723311, 0.9952046603199387, 0.9978844433112538, 0.9970733186941632, 0.9839793973943374, 0.9985602957710157, 0.9898248467893513, 0.9934845022348463, 0.9963294332780005, 0.989539734801525, 0.9989504673373727, 0.9981018890195538, 0.9878480748329147, 0.9896603635689731, 0.993965790570887, 0.9927316184746134, 0.99548333117915, 0.990592761722222, 0.9929643717511187, 0.9971905953066406, 0.9972015383352851, 0.9984373712446374, 0.9874947695233636, 0.9915345588546723, 0.9984510139952985, 0.9932499496307378, 0.9850643502577359, 0.9962023741133097, 0.992959564233347, 0.9913

In [ ]:
unlabel_1=unlabel_1.loc[[0, 2, 3, 4, 5, 6, 8, 10, 11, 12, 14, 16, 17, 19, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 34, 35, 37, 39, 40, 43, 48, 52, 56, 59, 62, 69, 70, 76, 77, 78, 80, 84, 86, 87, 88, 89, 91, 93, 94, 95, 96, 98, 99, 100],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
2,525,85 22 sw 140 wherein accused acting upon advic...,Rule/Law/Holding
3,2199,proper instruction court murder malice murder ...,Facts
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts


In [ ]:
frame_1 = [high_kappa,unlabel_1]
train_1 = pd.concat(frame_1)
len(train_1)

3005

In [ ]:
x_train_1 = tfidf_vect.transform(train_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train_1['Target'],test_size=0.2,random_state=42)
support =RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.64891846921797
                  precision    recall  f1-score   support

        Analysis       0.51      0.29      0.37        76
      Conclusion       0.80      0.41      0.54        39
           Facts       0.70      0.86      0.78       303
         Invalid       0.50      0.58      0.54        99
           Issue       0.76      0.44      0.56        43
Rule/Law/Holding       0.54      0.34      0.42        41

        accuracy                           0.65       601
       macro avg       0.64      0.49      0.53       601
    weighted avg       0.64      0.65      0.63       601



In [ ]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 3, 3, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2, 3, 2, 2, 2, 5, 3, 3, 4, 2]
[0.9985857863507207, 0.9988553501422813, 0.982690559042525, 0.9949406627188204, 0.9983676686058834, 0.9984460814163816, 0.9975990108565786, 0.9988473480898981, 0.9909795958392604, 0.9970181783322604, 0.9928168147027489, 0.9981036570855766, 0.9934880674844162, 0.993371444407133, 0.9981633529939422, 0.990076676722561, 0.9887926195124224, 0.9961107244306943, 0.9988553501422813, 0.9963509224854936, 0.9988402887377359, 0.9912268597462185, 0.9988641861537947, 0.9955253328417015, 0.9909230330605042, 0.9981000911455021, 0.9951410865031917, 0.99293568676621, 0.9986651380752763, 0.9965091427579195, 0.9987894689812928, 0.9988553501422813, 0.9964911860849094, 0.9982931451968758, 0.997084122629419, 0.9845363420916327, 0.9973727499136718, 0.9977206387981623, 0.9969398225196927, 0.982856420136354, 0.9986319602704294, 0.9966920171797753, 0.9980291407468256

In [ ]:
unlabel_2 = unlabel_2.loc[[1, 2, 3, 5, 6, 7, 8, 12, 13, 15, 17, 20, 22, 27, 29, 32, 34, 35, 37, 40, 42, 44, 46, 49, 50, 54, 55, 56, 59, 60, 63, 64, 66, 69, 71, 77, 78, 79, 80, 81, 83, 86, 88, 91, 93, 94, 95, 96, 99],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
5,2278,evidence state appellant shows appellant alleg...,Analysis
6,570,appellant performed work asked 7500 money ther...,Facts


In [ ]:
frame_2 = [train_1,unlabel_2]
train_2 = pd.concat(frame_2)
len(train_2)

3054

In [ ]:
x_train_2 = tfidf_vect.transform(train_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train_2['Target'],test_size=0.23,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.631578947368421
                  precision    recall  f1-score   support

        Analysis       0.45      0.24      0.31       102
      Conclusion       0.76      0.37      0.50        43
           Facts       0.68      0.89      0.77       358
         Invalid       0.50      0.54      0.52       107
           Issue       0.70      0.36      0.48        44
Rule/Law/Holding       0.59      0.20      0.30        49

        accuracy                           0.63       703
       macro avg       0.61      0.44      0.48       703
    weighted avg       0.62      0.63      0.60       703



In [ ]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 3, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2]
[0.9922614356121064, 0.9976381782347309, 0.9971228445392055, 0.9962899445500281, 0.9985480961972752, 0.9981978022044888, 0.9901565939759085, 0.9984495479466207, 0.9986056283073123, 0.9985814545768097, 0.9986410047045391, 0.9936169260959541, 0.9981689998258417, 0.9961347056056418, 0.9975215984362475, 0.9981611800802619, 0.9984494177630318, 0.9950298400154275, 0.9985832146261281, 0.9939050172540582, 0.9942625916439776, 0.9971673327840191, 0.9932156634253618, 0.9984306966088616, 0.9961864890771935, 0.993608717246249, 0.992484091002171, 0.9968780468716459, 0.9982375741179295, 0.9942873279941792, 0.9976381782347309, 0.996394001607724, 0.9984854212954656, 0.9980541367797187, 0.9976691994812112, 0.9981873090481768, 0.980613394159308, 0.9979396787411763, 0.9983258921368346, 0.9984192029743482, 0.98356

In [ ]:
unlabel_3=unlabel_3.loc[[2, 3, 4, 5, 7, 8, 10, 12, 14, 15, 17, 18, 20, 24, 25, 26, 27, 28, 30, 31, 33, 34, 35, 36, 39, 40, 41, 42, 43, 44, 46, 48, 49, 50, 51, 52, 53, 54, 56, 63, 64, 65, 67, 68, 70, 71, 72, 73, 74, 75, 78, 79, 80, 83, 86, 87, 88, 89, 90, 91, 92, 95, 96, 97, 98],:]
frame_3 = [train_2,unlabel_3]
train_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train_3['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6490384615384616
                  precision    recall  f1-score   support

        Analysis       0.53      0.30      0.38        83
      Conclusion       0.67      0.42      0.52        33
           Facts       0.70      0.91      0.79       320
         Invalid       0.57      0.50      0.53       110
           Issue       0.41      0.35      0.38        31
Rule/Law/Holding       0.62      0.17      0.27        47

        accuracy                           0.65       624
       macro avg       0.58      0.44      0.48       624
    weighted avg       0.63      0.65      0.62       624



In [ ]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 3, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 5, 2, 2, 2]
[0.9979299913223223, 0.9852902701639913, 0.986393163602167, 0.9978576856340097, 0.9984961360578463, 0.9932651981703069, 0.998288554446049, 0.9983731454990298, 0.9894824081352985, 0.9976003631185311, 0.9928600934964705, 0.9985457657141755, 0.9983288449463872, 0.9971968678095177, 0.9966472630941589, 0.9893766535780859, 0.9980548586892226, 0.99825838494449, 0.9986570575194331, 0.9967030792676701, 0.9979820026235525, 0.9978620839265031, 0.998124880529528, 0.9982240940777104, 0.989146914377226, 0.9981734877059495, 0.9954052437259541, 0.9974912840727318, 0.9962517930131919, 0.996414595441755, 0.9975752121310858, 0.9934409358218277, 0.9962145216511029, 0.980767060485569, 0.9969991314878924, 0.9963908551225417, 0.9976448362467627, 0.9969286443174681, 0.9974194943568747, 0.9979763843516795, 0.994415514756827, 0.993192643340505, 

In [ ]:
unlabel_4=unlabel_4.loc[[0, 1, 3, 4, 7, 8, 10, 11, 13, 15, 16, 18, 20, 21, 22, 28, 29, 30, 31, 32, 34, 35, 37, 38, 39, 40, 42, 49, 53, 55, 56, 57, 58, 60, 63, 66, 67, 69, 70, 71, 73, 74, 75, 79, 82, 83, 85, 87, 88, 89, 90, 91, 94, 96, 97, 99],:]

In [ ]:
frame_4 = [train_3,unlabel_4]
train_4 = pd.concat(frame_4)
len(train_4)
x_train_4 = tfidf_vect.transform(train_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train_4['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.675990675990676
                  precision    recall  f1-score   support

        Analysis       0.55      0.41      0.47       112
      Conclusion       0.69      0.42      0.52        43
           Facts       0.74      0.87      0.80       472
         Invalid       0.55      0.56      0.55       138
           Issue       0.55      0.41      0.47        44
Rule/Law/Holding       0.48      0.22      0.31        49

        accuracy                           0.68       858
       macro avg       0.59      0.48      0.52       858
    weighted avg       0.66      0.68      0.66       858



In [ ]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 3, 3, 2, 3, 0, 4, 2, 3, 3, 2, 2, 3, 2, 2, 2, 2, 1, 2, 1, 2, 1, 3, 0, 2, 2, 2, 3, 3, 2, 3, 2, 2, 2, 1, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 3, 2, 3, 2]
[0.9984604199272886, 0.9972148147605716, 0.9969823697112321, 0.9950548996231554, 0.9925266169754109, 0.9813671933776537, 0.9911005006550797, 0.997113247289421, 0.9976991085689123, 0.9960939532637855, 0.9969060263836801, 0.9982338549018002, 0.9933461598218166, 0.9949361934023554, 0.9957943356310773, 0.9858770686233704, 0.9891232223348274, 0.998079596718194, 0.9982768649421223, 0.9974092029582352, 0.9827395996692854, 0.9806257673542759, 0.9803064463486815, 0.992708133111661, 0.9960071028596729, 0.9983493782826178, 0.9983772370130279, 0.9976646243100729, 0.9969823697112321, 0.9971581682640426, 0.9977504291879877, 0.9878234343399385, 0.9975779349049209, 0.9964460512345666, 0.998079596718194, 0.9964209465965768, 0.9948419383695992, 0.995552365764093, 0.9964919629008752, 0.9948750360072839, 0.9802368938494382, 0.9985303300

In [ ]:
unlabel_5 = unlabel_5.loc[[3, 4, 5, 7, 8, 13, 15, 17, 19, 20, 22, 23, 24, 26, 27, 28, 30, 31, 32, 36, 40, 41, 42, 47, 50, 52, 53, 54, 55, 56, 57, 59, 60, 63, 64, 67, 68, 69, 73, 78, 79, 82, 85, 86, 90, 91, 92, 94, 95, 102, 106, 107, 111, 115, 116, 117, 118],:]

In [ ]:
frame_5 = [train_4,unlabel_5]
train_5 = pd.concat(frame_5)
len(train_5)
x_train_5 = tfidf_vect.transform(train_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train_5['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6568778979907264
                  precision    recall  f1-score   support

        Analysis       0.47      0.27      0.35        88
      Conclusion       0.74      0.49      0.59        35
           Facts       0.73      0.88      0.79       345
         Invalid       0.49      0.59      0.53        97
           Issue       0.54      0.42      0.47        31
Rule/Law/Holding       0.71      0.24      0.35        51

        accuracy                           0.66       647
       macro avg       0.61      0.48      0.51       647
    weighted avg       0.65      0.66      0.63       647



In [ ]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.7159309021113244


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.7208064689230361, 0.5958961464897966, 0.6382871835135534, None)
